## Проектна робота: Фінансовий аналітичний пайплайн

### _Суть проєктного завдання_
Розробити дата-пайплайн для фінансових даних за архітектурою Medallion (Bronze–Silver–Gold) у середовищі Databricks, використовуючи дані з Yahoo Finance через бібліотеку yfinance.

Архітектура проєкту: Medallion Architecture
1.	Bronze (raw): необроблені дані, збережені з yfinance.
2.	Silver (cleansed): очищені, нормалізовані, без дублікатів, готові для аналітики.
3.	Gold (aggregated): підсумовані таблиці для звітів — добові, тижневі, місячні метрики.


### 1.	Завантажити сирі фінансові дані з Yahoo Finance для кількох компаній (Bronze Layer).

_Вибираю дані по Industries in Software – Infrastructure Sector:
MSFT Microsoft Corporation,
NVDA NVIDIA Corporation,
AAPL Apple Inc.,
ORCL Oracle Corporation,
IBM International Business Machines Corporation_

In [0]:
%pip install yfinance

Python interpreter will be restarted.
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
  ERROR: Command errored out with exit status 1:
   command: /local_disk0/.ephemeral_nfs/envs/pythonEnv-3d0c61d3-f827-4560-91a6-0ac4f1df75b6/bin/python /local_disk0/.ephemeral_nfs/envs/pythonEnv-3d0c61d3-f827-4560-91a6-0ac4f1df75b6/lib/python3.9/site-packages/pip/_vendor/pep517/in_process/_in_process.py get_requires_for_build_wheel /tmp/tmpe9v0oh9t
       cwd: /tmp/pip-install-aa1768kb/peewee_aac35dcca02344a7a6a0cc822c383fa2
  Complete output (38 lines):
  
  Error compiling Cython file:
  ------------------------------------------------------------
  ...
  
  
  # We define an xConnect function, but leave xCreate NULL so that the
  # table-function can be called eponymously.
  cdef int pwConnect(sqlite3 *db, void *pAux, int argc, cons

Цей код для завантаження даних через yfinance в community.cloud.databricks видає помилку - блокування через надмірну кількість запитів (Edge: Too Many Requests)

import yfinance as yf
import pandas as pd
###### Список компаній
tickers = ["MSFT", "NVDA", "AAPL", "ORCL", "IBM"]
###### Завантаження фінансових даних
def get_stock_data(tickers, start, end):
    data = []
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        df = stock.history(start=start, end=end).reset_index()
        df["Ticker"] = ticker  #колонка з тикером компанії
        data.append(df)
    return pd.concat(data, ignore_index=True)  # Об'єднуємо всі дані в один DataFrame
###### Отримуємо дані
df_pandas = get_stock_data(tickers, start="2025-06-01", end="2025-06-05")

Тому дані отримано через скрипт Notebook в Datalore та збережено в файл .csv для подальшої роботи в Databricks.
Файл завантажено - /FileStore/tables/1_yahoofinance_data.csv

In [0]:
#Завантаження даних
df_work = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/FileStore/tables/1_yahoofinance_data.csv")
display(df_work)

Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker
2025-01-02T05:00:00.000+0000,423.90026204092493,424.43820240788705,413.2611726539346,416.97686767578125,16896500,0.0,0.0,MSFT
2025-01-03T05:00:00.000+0000,419.4672816518615,422.40599553232005,417.93320162542307,421.7286071777344,16662900,0.0,0.0,MSFT
2025-01-06T05:00:00.000+0000,426.3607841744266,432.6565862275622,423.85044657634234,426.21136474609375,20573600,0.0,0.0,MSFT
2025-01-07T05:00:00.000+0000,427.356962317073,429.00063686122985,419.18835553907815,420.7523498535156,18139100,0.0,0.0,MSFT
2025-01-08T05:00:00.000+0000,421.8381972488523,425.33476415419165,419.9255676122581,422.9339904785156,15054600,0.0,0.0,MSFT
2025-01-10T05:00:00.000+0000,423.0036976315441,423.0833778600729,413.4304875422897,417.345458984375,20201100,0.0,0.0,MSFT
2025-01-13T05:00:00.000+0000,413.6496498082875,416.8971739621178,410.71096634455273,415.5921936035156,17604800,0.0,0.0,MSFT
2025-01-14T05:00:00.000+0000,416.2098216251219,418.13242259689986,409.1469793562449,414.0780334472656,16935900,0.0,0.0,MSFT
2025-01-15T05:00:00.000+0000,417.52478252697114,426.51022596326726,416.6680604251186,424.6772766113281,19637800,0.0,0.0,MSFT
2025-01-16T05:00:00.000+0000,427.0581034768278,427.8450559110229,422.76461308505725,422.953857421875,15300000,0.0,0.0,MSFT


In [0]:
from pyspark.sql.functions import col

# Очищення назв колонок (замінюємо пробіли та спецсимволи)
df_work = df_work.select([col(c).alias(c.replace(" ", "_").replace(";", "").replace("(", "").replace(")", "").replace("=", "").replace("\n", "").replace("\t", "").replace("{", "").replace("}", "")) for c in df_work.columns])

In [0]:
# Збереження даних у Delta формат
df_work.write.format("delta").mode("overwrite").save("/FileStore/tables/bronze_yahoofinance_delta")

створення схеми:
CREATE DATABASE yahoo_data

In [0]:
%sql
SHOW DATABASES

databaseName
default
yahoo_data


In [0]:
%sql
USE yahoo_data;

In [0]:
#Створення Delta Table для сирих даних
df_work.write.format("delta").mode("overwrite").saveAsTable("yahoo_data.finance_bronze")

In [0]:
%sql
SELECT * FROM finance_bronze LIMIT 5;

Date,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Ticker
2025-01-02T05:00:00.000+0000,423.90026204092493,424.43820240788705,413.2611726539346,416.97686767578125,16896500,0.0,0.0,MSFT
2025-01-03T05:00:00.000+0000,419.4672816518615,422.40599553232005,417.93320162542307,421.7286071777344,16662900,0.0,0.0,MSFT
2025-01-06T05:00:00.000+0000,426.3607841744266,432.6565862275622,423.85044657634234,426.21136474609375,20573600,0.0,0.0,MSFT
2025-01-07T05:00:00.000+0000,427.356962317073,429.00063686122985,419.18835553907815,420.7523498535156,18139100,0.0,0.0,MSFT
2025-01-08T05:00:00.000+0000,421.8381972488523,425.33476415419165,419.9255676122581,422.9339904785156,15054600,0.0,0.0,MSFT
